In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import squarify

import plotly
plotly.tools.set_credentials_file(username='slhopy', api_key='lY5b6WqEHD7QKamOtxLN')
import plotly.plotly as py
import plotly.graph_objs as go

from domaintree import DomainTree

In [3]:
domain_df = pd.read_csv('domain.csv')
domain_df.fillna(method='ffill', inplace=True)
domainTree = DomainTree(domain_df)
del domainTree.tree.child[6]
del domainTree.tree.child[5]
del domainTree.tree.child[4]
del domainTree.tree.child[2]
del domainTree.tree.child[1]

In [4]:
domain1_names =[x.name for x in domainTree.tree.child]
domain1_names

['다항식', '도형의 방정식']

In [5]:
domain2_names = []
for domain1 in domainTree.tree.child:
    for domain2 in domain1.child:
        domain2_names.append(domain2.name)
domain2_names

['다항식의 연산',
 '인수분해',
 '항등식과 미정계수',
 '나머지 정리',
 '평면좌표',
 '직선의 방정식',
 '원의 방정식',
 '도형의 이동']

In [6]:
domain3_names = []
for domain1 in domainTree.tree.child:
    for domain2 in domain1.child:
        for domain3 in domain2.child:
            domain3_names.append(domain3.name)
domain3_names

['다항식의 정리',
 '다항식의 덧셈, 뺄셈',
 '다항식의 곱셈, 나눗셈',
 '곱셈공식',
 '인수분해의 기본 공식',
 '기본 공식의 활용',
 '다항식의 최대공약수와 최소공배수',
 '항등식의 성질과 미정계수법',
 '다항식의 나눗셈과 항등식',
 '나머지 정리',
 '인수정리와 고차식의 인수분해',
 '두 점 사이의 거리',
 '선분의 내분점과 외분점',
 '좌표와 자취',
 '방정식의 그래프',
 '두 직선의 위치 관계',
 '직선의 방정식',
 '정점을 지나는 직선',
 '점과 직선 사이의 거리',
 '자취 문제(직선)',
 '원의 방정식',
 '원과 직선의 위치 관계',
 '두 원의 위치 관계',
 '자취 문제(원)',
 '평행이동',
 '대칭이동']

In [7]:
n_domain = []
for d1 in domainTree.tree.child:
    domain1 = []
    for d2 in d1.child:
        domain2 = []
        for d3 in d2.child:
            domain2.append([1])
        domain1.append(domain2)
    n_domain.append(domain1)

In [8]:
rect2_heights = []
for d1 in n_domain:
    rect2_heights.append([len(x) for x in d1])
rect2_heights

[[4, 3, 2, 2], [3, 6, 4, 2]]

In [9]:
rect1_heights = [sum(x) for x in rect2_heights]
rect1_heights

[11, 15]

In [69]:
def make_rects(rect1_heights, rect2_heights):
    dx = 30
    
    rect1s = []
    x = 0.
    y = 0.
    for h in rect1_heights:
        rect1s.append({'x':x, 'y':y, 'dx':dx, 'dy':h})
        y += h
    
    def flatten(x):
        import collections
        if isinstance(x, collections.Iterable):
            return [a for i in x for a in flatten(i)]
        else:
            return [x]
        
    rect2_heights = flatten(rect2_heights)
    rect2s = []
    x = dx
    y = 0.
    for h in rect2_heights:
        rect2s.append({'x':x, 'y':y, 'dx':dx, 'dy':h})
        y += h
    
    rect3s = []
    x = dx * 2
    y = 0.
    for _ in range(sum(rect2_heights)):
        rect3s.append({'x':x, 'y':y, 'dx':dx, 'dy':1})
        y += 1
    
    return rect1s + rect2s + rect3s

In [70]:
rects = make_rects(rect1_heights, rect2_heights)

In [71]:
import matplotlib
from matplotlib import cm

magma_cmap = matplotlib.cm.get_cmap('RdBu')
viridis_cmap = matplotlib.cm.get_cmap('viridis')

viridis_rgb = []
magma_rgb = []
norm = matplotlib.colors.Normalize(vmin=0, vmax=255)

def matplotlib_to_plotly(cmap, pl_entries):
    h = 1.0/(pl_entries-1)
    pl_colorscale = []

    for k in range(pl_entries):
        C = list(map(np.uint8, np.array(cmap(k*h)[:3])*255))
        pl_colorscale.append([k*h, 'rgb'+str((C[0], C[1], C[2]))])

    return pl_colorscale

magma = matplotlib_to_plotly(magma_cmap, 14)
viridis = matplotlib_to_plotly(viridis_cmap, 10)
def make_colors(dt):

    for d1 in dt.tree.child:
        d1.color = 0
        for d2 in d1.child:
            d2.color = 0
            for d3 in d2.child:
                d3.color = np.random.randint(2, 12)
                d2.color += d3.color
            d2.color //= len(d2.child)
            d1.color += d2.color
        d1.color //= len(d1.child)
    
    d1_colors = [x.color for x in dt.tree.child]
    d2_colors = []
    for d1 in dt.tree.child:
        for d2 in d1.child:
            d2_colors.append(d2.color)
    d3_colors = []
    for d1 in dt.tree.child:
        for d2 in d1.child:
            for d3 in d2.child:
                d3_colors.append(d3.color)
    colors = d1_colors + d2_colors + d3_colors
    
    for i in range(len(colors)):
        colors[i] = magma[colors[i]][1]
    return colors

colors = make_colors(domainTree)

In [72]:
x = 0.
y = 0.
width = 200.
height = 200.

domain_names = domain1_names + domain2_names + domain3_names

shapes = []
annotations = []
counter = 0

for r in rects:
    shapes.append( 
        dict(
            type = 'rect', 
            x0 = r['x'], 
            y0 = r['y'], 
            x1 = r['x']+r['dx'], 
            y1 = r['y']+r['dy'],
            line = dict( width = 2 ),
            fillcolor = colors[counter]
        ) 
    )
    annotations.append(
        dict(
            x = r['x']+(r['dx']/2),
            y = r['y']+(r['dy']/2),
            text = domain_names[counter],
            showarrow = False
        )
    )
    counter = counter + 1
layout = dict(
    height=1000,
    width=900,
    xaxis=dict(showgrid=False,zeroline=False, visible=False),
    yaxis=dict(showgrid=False,zeroline=False, autorange='reversed', visible=False),
    shapes=shapes,
    annotations=annotations,
    hovermode='closest'
)
# With hovertext
# figure = dict(data=[trace0], layout=layout)

# Without hovertext
figure = dict(data=[go.Scatter()], layout=layout)
py.iplot(figure, filename='domain_tree_map')

In [73]:
def make_label_rects():
    dx = 10
    dy = 5
    rects = []
    x = 0.
    y = 0.

    for i in range(10):
        rects.append({'x':x, 'y':y, 'dx':dx, 'dy':dy})
        x += 10
    return rects
rects = make_label_rects()

In [77]:
x = 0.
y = 0.
width = 200.
height = 200.

colors = np.array(magma)[2:12, 1]

shapes = []
annotations = []
counter = 0



for r in rects:
    shapes.append( 
        dict(
            type = 'rect', 
            x0 = r['x'], 
            y0 = r['y'], 
            x1 = r['x']+r['dx'], 
            y1 = r['y']+r['dy'],
            line = dict( width = 2 ),
            fillcolor = colors[counter]
        ) 
    )
    annotations.append(
        dict(
            x = r['x']+(r['dx']/2),
            y = r['y']+(r['dy']/2),
            text = '',
            showarrow = False
        )
    )
    counter = counter + 1
layout = dict(
    height=220,
    width=800,
    title='correct rate heat map',
    xaxis=dict(showgrid=False,zeroline=False, dtick=10, ticksuffix='%'),
    yaxis=dict(showgrid=False,zeroline=False, autorange='reversed', visible=False),
    shapes=shapes,
    annotations=annotations,
    hovermode='closest'
)
# With hovertext
# figure = dict(data=[trace0], layout=layout)

# Without hovertext
figure2 = dict(data=[go.Scatter()], layout=layout)
py.iplot(figure2, filename='domain_tree_map_label')

In [78]:
py.iplot(figure, filename='domain_tree_map')